In [3]:
%pip install matplotlib

  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached pyparsing-3.2.3-py3-none-any.whl.metadata (5.0 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 47.6 MB/s eta 0:00:00
Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 51.5 MB/s eta 0:00:00
Using cached pyparsing-3.2.3-py3-none-any.whl (111 kB)

[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
import deepdoctection as dd
from IPython.core.display import HTML
from matplotlib import pyplot as plt

analyzer = dd.get_dd_analyzer()  # instantiate the built-in analyzer similar to the Hugging Face space demo

df = analyzer.analyze(path = "./data/Gascoigne_CC_tiny2.pdf")  # setting up pipeline
df.reset_state()                 # Trigger some initialization

doc = iter(df)
page = next(doc) 

image = page.viz(show_figures=True, show_residual_layouts=True)
plt.figure(figsize = (25,17))
plt.axis('off')
plt.imshow(image)

/Users/bigo/.pyenv/versions/3.12.7/envs/timbergem/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[0713 14:17.08 @dd.py:129]  INF  Config: 
 {'DEVICE': device(type='mps'),
 'LANGUAGE': None,
 'LAYOUT_LINK': {'CHILD_CATEGORIES': [<LayoutType.CAPTION>],
                 'PARENTAL_CATEGORIES': [<LayoutType.FIGURE>, <LayoutType.TABLE>]},
 'LAYOUT_NMS_PAIRS': {'COMBINATIONS': [[<LayoutType.TABLE>, <LayoutType.TITLE>],
                                       [<LayoutType.TABLE>, <LayoutType.TEXT>],
                                       [<LayoutType.TABLE>, <LayoutType.KEY_VALUE_AREA>],
                                       [<LayoutType.TABLE>, <LayoutType.LIST_ITEM>],
                                       [<LayoutType.TABLE>, <LayoutType.LIST>],
                                       [<LayoutType.TABLE>, <L

63303680it [00:17, 3716941.21it/s]                              


RuntimeError: Error(s) in loading state_dict for RecognitionPredictor:
	size mismatch for model.linear.weight: copying a param with shape torch.Size([124, 256]) from checkpoint, the shape in current model is torch.Size([127, 256]).
	size mismatch for model.linear.bias: copying a param with shape torch.Size([124]) from checkpoint, the shape in current model is torch.Size([127]).

In [4]:
import fitz  # PyMuPDF
import os

pdf_path = "/Users/bigo/Projects/timbergem/data/Gascoigne_CC_tiny2.pdf"
output_dir = "data"

os.makedirs(output_dir, exist_ok=True)

doc = fitz.open(pdf_path)
page = doc[0]  # first page

# Debug: Check page properties
print(f"Page rotation: {page.rotation}")
print(f"Page rect: {page.rect}")
print(f"Page CropBox: {page.cropbox}")
print(f"Page MediaBox: {page.mediabox}")
print(f"Page transformation matrix: {page.transformation_matrix}")

img_count = 0
for img in page.get_images(full=True):
    xref = img[0]
    print(f"\n--- Image {img_count} ---")
    print(f"Image info: {img}")
    print(f"Image xref: {xref}")
    
    pix = fitz.Pixmap(doc, xref)
    
    # Debug: Check pixmap properties
    print(f"Pixmap size: {pix.width} x {pix.height}")
    print(f"Pixmap n (components): {pix.n}")
    print(f"Pixmap alpha: {pix.alpha}")
    print(f"Pixmap colorspace: {pix.colorspace}")
    print(f"Pixmap stride: {pix.stride}")
    
    # Get image transformation matrix from page
    image_dict = doc.extract_image(xref)
    print(f"Image dictionary keys: {image_dict.keys()}")
    print(f"Image ext: {image_dict.get('ext', 'unknown')}")
    
    img_ext = "png" if pix.alpha else "jpg"
    img_filename = f"{output_dir}/page1_img{img_count}.{img_ext}"
    
    if pix.n < 5:  # GRAY or RGB
        pix.save(img_filename)
    else:  # CMYK: convert to RGB first
        pix1 = fitz.Pixmap(fitz.csRGB, pix)
        pix1.save(img_filename)
        pix1 = None
    pix = None
    img_count += 1

print(f"\nExtracted {img_count} images")
doc.close()

Page rotation: 0
Page rect: Rect(0.0, 0.0, 2592.0, 1728.0)
Page CropBox: Rect(0.0, 0.0, 2592.0, 1728.0)
Page MediaBox: Rect(0.0, 0.0, 2592.0, 1728.0)
Page transformation matrix: Matrix(1.0, 0.0, 0.0, -1.0, 0.0, 1728.0)

--- Image 0 ---
Image info: (7, 0, 908, 294, 8, 'ICCBased', '', 'Im1', 'FlateDecode', 0)
Image xref: 7
Pixmap size: 908 x 294
Pixmap n (components): 3
Pixmap alpha: 0
Pixmap colorspace: Colorspace(CS_RGB) - ICCBased(RGB,sRGB IEC61966-2.1)
Pixmap stride: 2724
Image dictionary keys: dict_keys(['width', 'height', 'ext', 'colorspace', 'xres', 'yres', 'bpc', 'size', 'image', 'smask', 'cs-name'])
Image ext: png

--- Image 1 ---
Image info: (8, 0, 901, 302, 8, 'ICCBased', '', 'Im2', 'FlateDecode', 0)
Image xref: 8
Pixmap size: 901 x 302
Pixmap n (components): 3
Pixmap alpha: 0
Pixmap colorspace: Colorspace(CS_RGB) - ICCBased(RGB,sRGB IEC61966-2.1)
Pixmap stride: 2703
Image dictionary keys: dict_keys(['width', 'height', 'ext', 'colorspace', 'xres', 'yres', 'bpc', 'size', 'image